# Chapter - 8 : Backpropagation

#### Backpropagation

<img src="backPropFormula.png"/>

In [1]:
# We will write code for backpropagation method
# Recall using CHAIN RULE, we find derivative of a function w.r.t the function inside it.
# We store this result, called gradient, and use it to multiply to the derivative to previous
# layer's output.

# For simplicity we are going to consider that the gradient we recieved from the
# next layer is 1, since multiplying with one wont change anything

# Let's code now

# FORWARD PASS
x = [1.0, -2.0, 3.0]    # inputs
w = [-3.0, -1.0, 2.0]   # weights
b = 1.0                 # bias

xw0 = x[0] * w[0]
xw1 = x[1] * w[1]
xw2 = x[2] * w[2]

# output of dense layer
sum = xw0 + xw1 + xw2 + b

# ReLU function
y = max(sum, 0)

# BACKWARD PASS

# The derivative from previous layer
dvalue = 1.0

# One important thing to note here is that the derivative of ReLU function is
# 1 if the input is greater than 1 else 0
drelu_dsum = dvalue * (1.0 if sum > 0 else 0.0)
print(drelu_dsum)

# Another important thing to note is that the partial derivative
# of a sum is always 1 no matter the inputs

# for x0w0 pair
dsum_dmulxw0 = 1
drelu_dmulxw0 = drelu_dsum * dsum_dmulxw0

# for x1w1 pair
dsum_dmulxw1 = 1
drelu_dmulxw1 = drelu_dsum * dsum_dmulxw1

# for x2w2 pair
dsum_dmulxw2 = 1
drelu_dmulxw2 = drelu_dsum * dsum_dmulxw2

# for b
dsum_db = 1
drelu_db = drelu_dsum * dsum_db

print(dsum_dmulxw0, dsum_dmulxw1, dsum_dmulxw2, drelu_db)

# Continuing with the backpropagation
# One more important thing to note here is that the partial derivative of
# a product is the value with which it is being multiplied.
# For eg.
# d(x*y)/d(x) = y
# d(x*y)/d(y) = x
# Therefore, partial derivative of the first weighted-input w.r.t the input equals the weight

# with respect to x values
dmulxw0_dx0 = w[0]
drelu_dx0 = drelu_dmulxw0 * dmulxw0_dx0

dmulxw1_dx1 = w[1]
drelu_dx1 = drelu_dmulxw1 * dmulxw1_dx1

dmulxw2_dx2 = w[2]
drelu_dx2 = drelu_dmulxw2 * dmulxw2_dx2

# with respect to w values (weights)
dmulxw0_dw0 = x[0]
drelu_dw0 = drelu_dmulxw0 * dmulxw0_dw0

dmulxw1_dw1 = x[1]
drelu_dw1 = drelu_dmulxw1 * dmulxw1_dw1

dmulxw2_dw2 = x[2]
drelu_dw2 = drelu_dmulxw2 * dmulxw2_dw2
print(drelu_dx0, drelu_dx1, drelu_dx2, drelu_dw0, drelu_dw1, drelu_dw2)


1.0
1 1 1 1.0
-3.0 -1.0 2.0 1.0 -2.0 3.0


<img src="./drelu_dz_final1.png" height="300px"/>

In [2]:
# The above completes out code for backpropagation for a single neuron.
# We can optimize the code a bit

drelu_dx0 = drelu_dmulxw0 * dmulxw0_dx0
# here
dmulxw0_dx0 = w[0]
# Therefore,
drelu_dx0 = drelu_dmulxw0 * w[0]

# Now
drelu_dmulxw0 = drelu_dsum * dsum_dmulxw0

# Then
drelu_dx0 = drelu_dsum * dsum_dmulxw0 * w[0]

# Here
dsum_dmulxw0 = 1

# Therefore
drelu_dx0 = drelu_dsum * 1 * w[0]
drelu_dx0 = drelu_dsum * w[0]

# where
drelu_dsum = dvalue * (1.0 if sum > 0 else 0.0)

# Finally
drelu_dx0 = dvalue * (1.0 if sum > 0 else 0.0) * w[0]


<img src="./optimized_form.png" height="300px"/>

In [3]:
# All together, the partial derivatives above, combined into a vector, make up our gradients. Our
# gradients could be represented as:

dx = [drelu_dx0, drelu_dx1, drelu_dx2]  # gradient vector for inputs
dw = [drelu_dw0, drelu_dw1, drelu_dw2]  # gradient vector for weights
db = drelu_db 


In [4]:
# Moving forward, we will discuss briefly about optimizers.

print("Original weights and biases: ", w, b)

# We then add a negative fraction of our gradients to these weights and biases
# and try to optimize it.
w[0] += -0.001 * dw[0]
w[1] += -0.001 * dw[1]
w[2] += -0.001 * dw[2]
b += -0.001 * db

# We are applying -ve fraction because we want to decrease the overall value
print("Weights after optimization: ", w, b)

# Now we'll observe the change that occurend due to out calculations
xw0 = x[0] * w[0]
xw1 = x[1] * w[1]
xw2 = x[2] * w[2]

sum = xw0 + xw1 + xw2 + b

# ReLU Activation Function
y = max(sum, 0)
print("New result: ", y)

# We’ve successfully decreased this neuron’s output from 6.000 to 5.985.


Original weights and biases:  [-3.0, -1.0, 2.0] 1.0
Weights after optimization:  [-3.001, -0.998, 1.997] 0.999
New result:  5.985


In [5]:
# Since now we've considered the case of a single neuron for the sake 
# of understanding.
# Now we'll move ahead and write code for a proper network with 
# array of inputs and weights.

# Now, let’s replace the current singular neuron with a layer of neurons. As opposed to a single
# neuron, a layer outputs a vector of values instead of a singular value. Each neuron in a layer
# connects to all of the neurons in the next layer. During backpropagation, each neuron from the
# current layer will receive a vector of partial derivatives the same way that we described for a
# single neuron. With a layer of neurons, it’ll take the form of a list of these vectors, or a 2D array.
# We know that we need to perform a sum, but what should we sum and what is the result supposed
# to be? Each neuron is going to output a gradient of the partial derivatives with respect to all of its
# inputs, and all neurons will form a list of these vectors. We need to sum along the inputs — the
# first input to all of the neurons, the second input, and so on. We’ll have to sum columns.

# To calculate the partial derivatives with respect to inputs, we need the weights — the partial
# derivative with respect to the input equals the related weight. This means that the array of
# partial derivatives with respect to all of the inputs equals the array of weights. Since this array is
# transposed, we’ll need to sum its rows instead of columns. To apply the chain rule, we need to
# multiply them by the gradient from the subsequent function.

# In the code to show this, we take the transposed weights, which are the transposed array of the
# derivatives with respect to inputs, and multiply them by their respective gradients (related to
# given neurons) to apply the chain rule. Then we sum along with the inputs. Then we calculate
# the gradient for the next layer in backpropagation. The “next” layer in backpropagation is the
# previous layer in the order of creation of the model.


In [34]:
# Backward Propagation

import numpy as np

# These are the gradients passed from the next layer.
# Note: gradient is a vector which has values equal to the
# neurons in the previous layer
# Eg. We have 3 neurons in our current layer, therefore, the gradients will
# contain 3 values.
dvalues = np.array([[1.0, 1.0, 1.0]])

# 3 neurons with 4 weights
weights = np.array([
    [0.2, 0.8, -0.5, 1],
    [0.5, -0.91, 0.26, -0.5],
    [-0.26, -0.27, 0.17, 0.87]
])

# We know that the derivative of inputs is the related weight.
# Therefore we transpose weights
# gradients for the current layer
inputs = weights.T
dinputs = np.dot(dvalues, inputs.T)

print("Gradients: ", dinputs)


Gradients:  [[ 0.44 -0.38 -0.07  1.37]]


In [37]:
# in case of batch-of-data
import numpy as np

# 3 batches of inputs
dvalues = np.array([
    [1.0, 1.0, 1.0],
    [2.0, 2.0, 2.0],
    [3.0, 3.0, 3.0],
])

# 3 neurons with 4 weights
weights = np.array([
    [0.2, 0.8, -0.5, 1],
    [0.5, -0.91, 0.26, -0.5],
    [-0.26, -0.27, 0.17, 0.87]
])

inputs = weights
dinputs = np.dot(dvalues, inputs)

print("Gradients w.r.t inputs:\n ", dinputs)


Gradients w.r.t inputs:
  [[ 0.44 -0.38 -0.07  1.37]
 [ 0.88 -0.76 -0.14  2.74]
 [ 1.32 -1.14 -0.21  4.11]]


In [40]:
# similary for dweights
import numpy as np

# 3 batches of inputs
dvalues = np.array([
    [1.0, 1.0, 1.0],
    [2.0, 2.0, 2.0],
    [3.0, 3.0, 3.0],
])

inputs = np.array([
    [1, 2, 3, 2.5],
    [2, 5, -1, 2],
    [-1.5, 2.7, 3.3, -0.8]
])

weights = inputs
dweights = np.dot(weights.T, dvalues)

print("Gradients w.r.t inputs:\n ", dweights)


Gradients w.r.t inputs:
  [[ 0.5  0.5  0.5]
 [20.1 20.1 20.1]
 [10.9 10.9 10.9]
 [ 4.1  4.1  4.1]]


In [41]:
# similarly for biases

# For the biases and derivatives with respect to them, the derivatives come from the sum operation
# and always equal 1, multiplied by the incoming gradients to apply the chain rule. Since gradients
# are a list of gradients (a vector of gradients for each neuron for all samples), we just have to sum
# them with the neurons, column-wise, along axis 0.

# in case of batch-of-data
import numpy as np

# 3 batches of inputs
dvalues = np.array([
    [1.0, 1.0, 1.0],
    [2.0, 2.0, 2.0],
    [3.0, 3.0, 3.0],
])

# 3 neurons with 4 weights
biases = np.array([[2, 3, 0.5]])

dinputs = np.sum(dvalues, axis=0, keepdims=True)

print("Gradients w.r.t biases:\n ", dinputs)


Gradients w.r.t biases:
  [[6. 6. 6.]]
